In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import operator
import csv
from collections import Counter
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [2]:
warnings.filterwarnings('ignore')

In [6]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [4]:
data = pd.read_csv(r'C:\Users\user\Desktop\Radius\September\1st Week\cses5.csv')

data = data.rename(columns={'E1006_NAM': 'country_name'})
data = data.rename(columns={'E2002': 'gender'})
data = data.rename(columns={'E2003': 'education'})
data = data.rename(columns={'E2006': 'employement'})
data = data.rename(columns={'E3024_4': 'closeness_party'})
data = data.rename(columns={'E4006': 'turnout'})
data = data.rename(columns={'E5075': 'women_in_parliament'})

data1 = data[['country_name', 'gender','education',
        'employement', 'closeness_party','turnout','women_in_parliament']]

df_dict = {ct: data1[data1.country_name== ct].reset_index(drop=True) for ct in data1['country_name'].unique()}

In [5]:
import ipywidgets as widgets

In [7]:
a = widgets.Dropdown(
    options=['0.05', '0.01', '0.001'],
    value='0.05',
    description='α level:',
    disabled=False,
    continuous_update=False

)

b = widgets.Dropdown(
    options=['Turkey', 'Switzerland'],
    value='Turkey',
    description='Country:',
    disabled=False,
    continuous_update=False
)


c = widgets.Dropdown(
    options=['one-tail', 'two-tail'],
    value='one-tail',
    description='Test:',
    disabled=False,
    continuous_update=False
)

def two_tail(country,tail,value):
    
    values_2 = np.array(df_dict[country]['turnout'])
    
    if tail == 'one-tail':
        if value == '0.05':
            mult = 1.645
        elif value == '0.01':
            mult = 2.33
        elif value == '0.001':
            mult = 3.1
        two_std_from_mean = np.mean(values_2) + np.std(values_2)*mult
        kde = stats.gaussian_kde(values_2)
        pos = np.linspace(np.min(values_2)-10, np.max(values_2)+10, 10000)
        plt.plot(pos, kde(pos), color='teal')

        shade = np.linspace(two_std_from_mean, np.max(values_2)+10, 300)
        plt.fill_between(shade, kde(shade), alpha=0.45, color='teal')

        plt.axvline(two_std_from_mean, 0, 1, color ='gray',linestyle='--')

        plt.title("Turnout Distribution in Switzerland for One-Tail Hypothesis Test", y=1.015, fontsize=15)
        plt.xlabel("turnout", labelpad=14)
        plt.ylabel("frequency of occurence", labelpad=14)
    
    elif tail == 'two-tail':
        if value == '0.05':
            mult = 1.96
        elif value == '0.01':
            mult = 2.575
        elif value == '0.001':
            mult = 3.29
        
        alpha_05_positive = np.mean(values_2) + np.std(values_2)*mult
        alpha_05_negative = np.mean(values_2) - np.std(values_2)*mult
        kde = stats.gaussian_kde(values_2)
        pos = np.linspace(np.min(values_2)-10, np.max(values_2)+10, 10000)

        plt.plot(pos, kde(pos), color='dodgerblue')

        shade = np.linspace(alpha_05_positive, np.max(values_2)+10, 300)
        plt.fill_between(shade, kde(shade), alpha=0.45, color='dodgerblue')

        shade2 = np.linspace(alpha_05_negative, np.min(values_2)-10, 300)
        plt.fill_between(shade2, kde(shade2), alpha=0.45, color='dodgerblue')

        plt.axvline(alpha_05_positive, 0, 1, color ='gray',linestyle='--')
        plt.axvline(alpha_05_negative, 0, 1, color ='gray',linestyle='--')

        plt.title("""Turnout Distribution in %s for Two-Tail Hypothesis Test"""%(country), y=1.015, fontsize=15)
        plt.xlabel("turnout", labelpad=14)
        plt.ylabel("frequency of occurence", labelpad=14);

widgets.interact(two_tail, value=a, tail=c, country=b);

interactive(children=(Dropdown(description='Country:', options=('Turkey', 'Switzerland'), value='Turkey'), Dro…

In [ ]:
!pip freeze > requirements.txt